In [ ]:
import pandas as pd 
import numpy as np 
import keras
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('forestfires.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
y = data.iloc[:,10]
y

In [ ]:
x = data.loc[:, ~data.columns.isin(['month','day','area'])]
x

In [ ]:
x = pd.get_dummies(x, columns=['size_category'])
x

In [ ]:
# Standardization
a = StandardScaler()
a.fit(x)
x_standardized = a.transform(x)
x_standardized

In [ ]:
x_standardized.shape

### Load required packages

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam

### Define the model

In [ ]:
def create_model(learning_rate,dropout_rate,activation_function,init,neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim=29,kernel_initializer=init,activation=activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2,input_dim=neuron1,kernel_initializer=init,activation=activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation='sigmoi'))

    adam = Adam(learning_rate= learning_rate)
    model.compile(loss='binary_crossentropy',optimizer=adam, metrics=['accuracy'])
    return model

### Create the model

In [ ]:
model = KerasClassifier(build_fn=create_model,verbose=0)

### Define grid search parameters

In [ ]:
batch_size = [10,20,40]
epochs = [10,50,100]
learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]
activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']
neuron1 = [4,8,16]
neuron2 = [2,4,8]

### make a dictionary of the grid search parameters

In [ ]:
param_grids = dict(batch_size = batch_size,epochs = epochs,learning_rate = learning_rate,dropout_rate = dropout_rate,
                   activation_function = activation_function,init = init,neuron1 = neuron1,neuron2 = neuron2)

### Build and fit the GridSearchCV

In [ ]:
grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(x_standardized,y)

### Summarize the result

In [ ]:
print(r'Best:{grid_result.best_score}, using:{grid_result.best_params_}')